### Libraries

In [1]:
import os
import sys

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

# (Optional) If you're working inside Jupyter
%load_ext autoreload
%autoreload 2
%matplotlib inline



In [2]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cpu


In [3]:
from data_processor_v2 import DataProcessorUpdated

processor = DataProcessorUpdated()

In [4]:
processor.load_and_clean_data()

2023-12-31 23:00:00


/ephnvme/weiliang/synthesis-data/energy-prediction/pytorch/data_processor_v2.py:41: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[time_col] = pd.to_datetime(df[time_col])


2024-01-01 23:00:00
2023-12-31 23:00:00
2023-12-31 23:00:00


In [5]:
processor.combine_all_data()

In [6]:
processor.df

,DAP_SystemLambda,SCED_system_lambda,Fuel_coal_and_lignite,Fuel_hydro,Fuel_nuclear,Fuel_power_storage,Fuel_solar,Fuel_wind,Fuel_natural_gas,Fuel_other,Load_load,delta_price
2019-01-01 00:00:00,NaN,13.837562,6116.419040,185.465296,3895.951940,0.0,0.0,14311.36445,12512.80815,2.534772,36951.0,NaN
2019-01-01 01:00:00,NaN,15.464869,6423.242360,186.667008,3894.973176,0.0,0.0,14298.52586,12434.13638,2.947464,37112.0,NaN
2019-01-01 02:00:00,NaN,15.487720,6309.280752,187.408832,3894.733152,0.0,0.0,14030.82875,12797.25831,-1.954544,37154.0,NaN
2019-01-01 03:00:00,NaN,15.770092,6416.671292,187.817564,3894.714576,0.0,0.0,13610.13937,13279.01803,1.887324,37283.0,NaN
2019-01-01 04:00:00,NaN,16.036085,6569.580884,186.990116,3892.748912,0.0,0.0,13414.14969,13585.26799,-0.201100,37817.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-01 19:00:00,23.1651,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-01 20:00:00,23.2113,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-01 21:00:00,21.3244,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-01 22:00:00,20.3351,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
processor.shift_dap()

In [8]:
processor.df

,DAP_SystemLambda,SCED_system_lambda,Fuel_coal_and_lignite,Fuel_hydro,Fuel_nuclear,Fuel_power_storage,Fuel_solar,Fuel_wind,Fuel_natural_gas,Fuel_other,Load_load,delta_price
2019-01-01 00:00:00,23.9250,13.837562,6116.419040,185.465296,3895.951940,0.0,0.0,14311.36445,12512.80815,2.534772,36951.0,NaN
2019-01-01 01:00:00,23.3140,15.464869,6423.242360,186.667008,3894.973176,0.0,0.0,14298.52586,12434.13638,2.947464,37112.0,NaN
2019-01-01 02:00:00,23.3475,15.487720,6309.280752,187.408832,3894.733152,0.0,0.0,14030.82875,12797.25831,-1.954544,37154.0,NaN
2019-01-01 03:00:00,23.0595,15.770092,6416.671292,187.817564,3894.714576,0.0,0.0,13610.13937,13279.01803,1.887324,37283.0,NaN
2019-01-01 04:00:00,25.2672,16.036085,6569.580884,186.990116,3892.748912,0.0,0.0,13414.14969,13585.26799,-0.201100,37817.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-01 19:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-01 20:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-01 21:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-01 22:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
processor.split_data(feature_columns=['DAP_SystemLambda', 'SCED_system_lambda'])

In [10]:
processor.standardize_data()

In [11]:
processor.shift_data()

In [12]:
x_train_mlp, x_val_mlp, x_test_mlp, y_train_mlp, y_val_mlp, y_test_mlp = processor.flatten()

In [13]:
x_train_mlp.shape

(26089, 336)

In [14]:
y_train_mlp.shape

(26089, 24)

In [15]:
len(x_train_mlp[1])

336

# Model Building

In [16]:
from MLP import MLPModel

new_model = MLPModel(
    input_shape=len(x_train_mlp[1]),    # 5 features
    output_shape=len(y_train_mlp[1]),   # 1 price per time step
    hidden_layers=[512, 512, 512]  # or [256, 256]
)

mlp_model = new_model.get_model()
mlp_model.to(device)


MLPModel(
  (model): Sequential(
    (0): Linear(in_features=336, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=512, bias=True)
    (5): ReLU()
    (6): Linear(in_features=512, out_features=24, bias=True)
  )
)

In [17]:
from ModelTrainer import ModelTrainer
from losses import fluctuation_loss  # or define it above in your notebook

trainer = ModelTrainer(
    model=mlp_model,
    features_training_data=x_train_mlp,
    target_training_data=y_train_mlp,
    features_eval_data=x_val_mlp,
    target_eval_data=y_val_mlp,
    device=device,
    loss_fn=lambda pred, target: fluctuation_loss(pred, target, alpha=0.2)
)


trainer.train(epochs=100, batch_size=32, patience=50, learning_rate=5e-4)


Epoch 1: Train Loss = 0.7272, Eval Loss = 0.2576
Epoch 2: Train Loss = 0.6206, Eval Loss = 0.2546
Epoch 3: Train Loss = 0.6067, Eval Loss = 0.2537
Epoch 4: Train Loss = 0.5516, Eval Loss = 0.2530
Epoch 5: Train Loss = 0.5426, Eval Loss = 0.2562
Epoch 6: Train Loss = 0.4771, Eval Loss = 0.2567
Epoch 7: Train Loss = 0.4621, Eval Loss = 0.2571
Epoch 8: Train Loss = 0.4530, Eval Loss = 0.2601
Epoch 9: Train Loss = 0.4061, Eval Loss = 0.2570
Epoch 10: Train Loss = 0.3661, Eval Loss = 0.2557
Epoch 11: Train Loss = 0.3380, Eval Loss = 0.2587
Epoch 12: Train Loss = 0.3378, Eval Loss = 0.2571
Epoch 13: Train Loss = 0.3272, Eval Loss = 0.2586
Epoch 14: Train Loss = 0.3582, Eval Loss = 0.2569
Epoch 15: Train Loss = 0.2832, Eval Loss = 0.2578
Epoch 16: Train Loss = 0.2729, Eval Loss = 0.2596
Epoch 17: Train Loss = 0.2595, Eval Loss = 0.2583
Epoch 18: Train Loss = 0.2660, Eval Loss = 0.2565
Epoch 19: Train Loss = 0.2586, Eval Loss = 0.2580
Epoch 20: Train Loss = 0.2415, Eval Loss = 0.2578
Epoch 21:

In [18]:
import optuna
from MLP import MLPModel
from ModelTrainer import ModelTrainer
from losses import fluctuation_loss

def objective(trial):
    # Suggest hyperparameters
    hidden_dim = trial.suggest_categorical("hidden_dim", [128, 256, 512])
    n_layers = trial.suggest_int("n_layers", 2, 4)
    lr = trial.suggest_loguniform("lr", 1e-5, 1e-3)
    alpha = trial.suggest_uniform("alpha", 0.1, 0.5)

    # Build hidden layers
    hidden_layers = [hidden_dim] * n_layers

    # Build model
    model = MLPModel(input_shape=len(x_train_mlp[1]), output_shape=len(y_train_mlp[1]), hidden_layers=hidden_layers).get_model().to(device)

    # Trainer
    trainer = ModelTrainer(
        model=model,
        features_training_data=x_train_mlp,
        target_training_data=y_train_mlp,
        features_eval_data=x_val_mlp,
        target_eval_data=y_val_mlp,
        device=device,
        loss_fn=lambda pred, target: fluctuation_loss(pred, target, alpha=alpha)
    )

    # Train for a few epochs
    trainer.train(epochs=50, batch_size=32, patience=10, learning_rate=lr)

    # Return final eval loss (or early stopping best)
    return trainer.history['eval_loss'][-1]


In [19]:

study = optuna.create_study(
    direction="minimize",
    pruner=optuna.pruners.HyperbandPruner()
)
study.optimize(objective, n_trials=50)


[I 2025-05-10 20:59:33,850] A new study created in memory with name: no-name-03e09cac-5f0a-4a99-ba44-868848864ed7


/ephnvme/weiliang/cache/tmpdir/ipykernel_3974192/1319189539.py:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform("lr", 1e-5, 1e-3)
/ephnvme/weiliang/cache/tmpdir/ipykernel_3974192/1319189539.py:11: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  alpha = trial.suggest_uniform("alpha", 0.1, 0.5)
[W 2025-05-10 20:59:37,056] Trial 0 failed with parameters: {'hidden_dim': 512, 'n_layers': 3, 'lr': 6.687772847470267e-05, 'alpha': 0.4387761646006991} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/ephnvme/weiliang/miniforge/envs/eng/lib/python3.10/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    va

Epoch 1: Train Loss = 0.8404, Eval Loss = 0.3011


KeyboardInterrupt: 

In [ ]:
print("Best hyperparameters:", study.best_trial.params)


In [ ]:
# predict

mlp_model.eval()
with torch.no_grad():
    y_pred = mlp_model(torch.tensor(x_test_mlp, dtype=torch.float32).to(device))


y_pred = y_pred.cpu().numpy()
errors = y_pred - y_test_mlp  # assuming y_test is numpy array
mse = np.mean(errors**2)
rmse = np.sqrt(mse)
mae = np.mean(np.abs(errors))

print(f"Test MAE: {mae:.4f}")
print(f"Test RMSE: {rmse:.4f}")

